# American Express Data Analysis

## Importing Basic Libraries

In [32]:
import numpy as np
import pandas as pd
import tensorflow as tf
import openpyxl

## Importing dataset

In [34]:
dataset  = pd.read_excel('dataset/AMEX_Credit.xlsx', engine='openpyxl')
X = dataset.iloc[:, 0:-1].values
y = dataset.iloc[:, -1].values

In [ ]:
print(X)
print(y)

## Importing LabelEncoder

### Encoding Gender

In [36]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
X[:, 2] = label_encoder.fit_transform(X[:, 2])

In [ ]:
print(X)

In [38]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
print(X)

## Spliting dataset into Test and Training Set

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [41]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## ANN Build Up

In [43]:
ann = tf.keras.models.Sequential()

### Adding 1st layer

In [44]:
ann.add(tf.keras.layers.Dense(units=5, activation='relu'))

### Adding 2nd layer

In [45]:
ann.add(tf.keras.layers.Dense(units=5, activation='relu'))

### Adding output layer

In [46]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### Compiling Model

In [47]:
ann.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

### Training Model

In [ ]:
ann.fit(X_train, y_train, batch_size=32, epochs=120)

### Predicting Sample Output

In [ ]:
print(ann.predict(sc.transform([[0.0, 1.0, 0.0, 501, 0, 32, 2, 0.0, 4, 1, 545501]])) > 0.5)

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

## Generating Confusion Matrix and Accuracy Score

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
ac = accuracy_score(y_test, y_pred)
print("Confusion Matrix", cm)
print("Accuracy Score", ac)